In [3]:
# import pandas

import pandas as pd

In [4]:
# importing data

df_basketcontents = pd.read_excel('./Basket Contents.xlsx')

df_stuffedbunny = pd.read_excel('./Stuffed Bunny Price List.xlsx')

df_treatsprice = pd.read_excel('./Treats Price List.xlsx')

In [5]:
df_basketcontents.head()

,Basket List,Purchase Year,Treat,Stuffed Animal
0,Basket A,2021,Chocolate Egg,Blue Bunny
1,Basket A,2022,Chocolate Egg,Blue Bunny
2,Basket A,2023,Chocolate Egg,Blue Bunny
3,Basket B,2021,Jelly Beans,Blue Bunny
4,Basket B,2022,Jelly Beans,Blue Bunny


In [6]:
df_stuffedbunny.head()

,Stuffed animal,Price,Year
0,Blue Bunny,6.49,2021
1,Blue Bunny,7.99,2022
2,Blue Bunny,10.99,2023
3,Pink Bunny,7.49,2021
4,Pink Bunny,7.99,2022


In [7]:
df_treatsprice.head()

,Treat,Price,Year
0,Chocolate Egg,9.99,2021
1,Chocolate Egg,10.99,2022
2,Chocolate Egg,12.99,2023
3,Jelly Beans,3.49,2021
4,Jelly Beans,3.99,2022


In [8]:
# merging df_basketcontents with stuffed bunny and treats price

df_merge_1 = df_basketcontents.merge(df_stuffedbunny, left_on = ['Purchase Year', 'Stuffed Animal'], right_on = ['Year', 'Stuffed animal'])

In [9]:
df_merge_2 = df_basketcontents.merge(df_treatsprice, left_on = ['Purchase Year', 'Treat'], right_on = ['Year', 'Treat'])

In [10]:
# unioning both dfs

df_union = pd.concat([df_merge_1, df_merge_2])

# selecting columns

df_union = df_union[['Basket List', 'Purchase Year', 'Stuffed Animal', 'Treat', 'Price']]

df_union.sort_values(by = ['Basket List', 'Purchase Year'])

,Basket List,Purchase Year,Stuffed Animal,Treat,Price
0,Basket A,2021,Blue Bunny,Chocolate Egg,6.49
0,Basket A,2021,Blue Bunny,Chocolate Egg,9.99
2,Basket A,2022,Blue Bunny,Chocolate Egg,7.99
2,Basket A,2022,Blue Bunny,Chocolate Egg,10.99
4,Basket A,2023,Blue Bunny,Chocolate Egg,10.99
4,Basket A,2023,Blue Bunny,Chocolate Egg,12.99
1,Basket B,2021,Blue Bunny,Jelly Beans,6.49
6,Basket B,2021,Blue Bunny,Jelly Beans,3.49
3,Basket B,2022,Blue Bunny,Jelly Beans,7.99
8,Basket B,2022,Blue Bunny,Jelly Beans,3.99


In [105]:
# summing total basket price per year

df_sum = df_union.groupby(['Basket List', 'Purchase Year'])['Price'].sum().reset_index(name = 'price')

# multiplying by 25 (25 of each basket combo)

df_summed = df_sum.copy()

df_summed['price'] = df_sum['price'] * 25

df_summed

,Basket List,Purchase Year,price
0,Basket A,2021,412.0
1,Basket A,2022,474.5
2,Basket A,2023,599.5
3,Basket B,2021,249.5
4,Basket B,2022,299.5
5,Basket B,2023,412.0
6,Basket C,2021,437.0
7,Basket C,2022,474.5
8,Basket C,2023,562.0
9,Basket D,2021,274.5


In [16]:
# pivoting data

df_pivot = pd.pivot(df_summed, 
        index = 'Basket List', 
        columns = 'Purchase Year', 
        values = 'price')

df_pivot

Purchase Year,2021,2022,2023
Basket List,,,
Basket A,412.0,474.5,599.5
Basket B,249.5,299.5,412.0
Basket C,437.0,474.5,562.0
Basket D,274.5,299.5,374.5


In [17]:
# subtracting YoY and finding percentage difference

def YoY (row):
    diff1 = round(((row.iloc[1] - row.iloc[0])/row.iloc[0]) * 100, 2)
    diff2 = round(((row.iloc[2] - row.iloc[1])/row.iloc[1]) * 100, 2)
    return {'2021-2022' : diff1, 
           '2022-2023' : diff2}

In [18]:
# applying function

dict = df_pivot.apply(YoY, axis = 1)

# converting to df

df_diff = pd.DataFrame(list(dict), index = df_pivot.index)

df_diff

,2021-2022,2022-2023
Basket List,,
Basket A,15.17,26.34
Basket B,20.04,37.56
Basket C,8.58,18.44
Basket D,9.11,25.04


In [109]:
# finding average price across years

df_avg = df_sum.groupby('Purchase Year')['price'].mean().reset_index()

# filtering to 2023

avg_price = df_avg.loc[df_avg['Purchase Year'] == 2023]['price']

avg_price

2    19.48
Name: price, dtype: float64

In [110]:
# finding avg YoY diff in 2022-2023

avg_yoy = df_diff['2022-2023'].mean()

avg_yoy

26.845

In [111]:
# calculating fundraising goal for 2024

round((avg_price * (1 + avg_yoy/100)) * 100, 2)

2    2470.94
Name: price, dtype: float64

# More efficient way to find YoY difference

In [112]:
df_summed

,Basket List,Purchase Year,price
0,Basket A,2021,412.0
1,Basket A,2022,474.5
2,Basket A,2023,599.5
3,Basket B,2021,249.5
4,Basket B,2022,299.5
5,Basket B,2023,412.0
6,Basket C,2021,437.0
7,Basket C,2022,474.5
8,Basket C,2023,562.0
9,Basket D,2021,274.5


In [113]:
# using shift

df_summed['suceeding_price'] = df_summed['price'].shift(1)

df_summed['YoY_Change'] = df_summed['suceeding_price'] - df_summed['price']

df_summed

# next, have to find a way to change purchase year title to e.g. 2021-2022

,Basket List,Purchase Year,price,suceeding_price,YoY_Change
0,Basket A,2021,412.0,NaN,NaN
1,Basket A,2022,474.5,412.0,-62.5
2,Basket A,2023,599.5,474.5,-125.0
3,Basket B,2021,249.5,599.5,350.0
4,Basket B,2022,299.5,249.5,-50.0
5,Basket B,2023,412.0,299.5,-112.5
6,Basket C,2021,437.0,412.0,-25.0
7,Basket C,2022,474.5,437.0,-37.5
8,Basket C,2023,562.0,474.5,-87.5
9,Basket D,2021,274.5,562.0,287.5


In [114]:
# replicating multi-row in pandas

def multi_row(group):
    results = []
    previous_price = pd.NA
    previous_year = pd.NA
    for index, row in group.iterrows():
        perc_change = pd.NA if pd.isna(previous_price) else round(((row['price'] - previous_price) / previous_price) * 100, 2)
        previous_price = row['price']
        year_range = pd.NA if pd.isna(previous_year) else f"{previous_year}-{row['Purchase Year']}"
        previous_year = row['Purchase Year']
        dict = ({'Basket List' : row['Basket List'],
                'range' : year_range,
                'YoY' : perc_change})
        results.append(dict)
        
    return results

In [116]:
# applying multi-row function

result = df_summed.groupby('Basket List').apply(multi_row).explode().reset_index(drop = True)

df_result = pd.json_normalize(result)

df_result

,Basket List,range,YoY
0,Basket A,<NA>,<NA>
1,Basket A,2021-2022,15.17
2,Basket A,2022-2023,26.34
3,Basket B,<NA>,<NA>
4,Basket B,2021-2022,20.04
5,Basket B,2022-2023,37.56
6,Basket C,<NA>,<NA>
7,Basket C,2021-2022,8.58
8,Basket C,2022-2023,18.44
9,Basket D,<NA>,<NA>


In [117]:
result

0     {'Basket List': 'Basket A', 'range': <NA>, 'Yo...
1     {'Basket List': 'Basket A', 'range': '2021-202...
2     {'Basket List': 'Basket A', 'range': '2022-202...
3     {'Basket List': 'Basket B', 'range': <NA>, 'Yo...
4     {'Basket List': 'Basket B', 'range': '2021-202...
5     {'Basket List': 'Basket B', 'range': '2022-202...
6     {'Basket List': 'Basket C', 'range': <NA>, 'Yo...
7     {'Basket List': 'Basket C', 'range': '2021-202...
8     {'Basket List': 'Basket C', 'range': '2022-202...
9     {'Basket List': 'Basket D', 'range': <NA>, 'Yo...
10    {'Basket List': 'Basket D', 'range': '2021-202...
11    {'Basket List': 'Basket D', 'range': '2022-202...
dtype: object